In [1]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
import composio_langchain

# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)

In [14]:
Initial_prompt = """You are an expert python developer. You will be given a description of a python function. 

You job is to estimate and extract the following information:

- What exactly does this python do. What is the detailed objective of the function. Please write 1-5 lines
- Suggest or extract the name of the the function
- What would be the inputs/arguements required into this function to make it work. Please all mentioned the type of each input
- WHat would be output produced by this input. Please mention the output type 
"""

In [15]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", streaming=True)

In [16]:
class FunctionInstructions(BaseModel):
    """Instructions for defining a python function"""
    objective: str = Field(description= "what does this pythion function do")
    name: str = Field(description="name of the python function")
    input : List[str] = Field(description= "what would be the input arguements to this function along with the types")
    output: List[str] = Field(description="what would be the output/return attributes for the function along with the types")
    code: str = Field(description="the final python code")

class CodebuilderState(BaseModel):
    """Instructions for defining a python function"""
    code: str = Field(description= "tailored code for the python function")


In [21]:
human_prompt = """
Create a python function that gets all my emails from gmail and filter them based on senders"
"""
def functional_analysis_node(state: FunctionInstructions):
  llm_with_structured_output = llm.with_structured_output(FunctionInstructions)
  functionalReport: FunctionInstructions = llm_with_structured_output.invoke(
      [SystemMessage(content=Initial_prompt)]+ [HumanMessage(content=human_prompt)])
  return {  "messages": [AIMessage(content="Generated JSON code!")],
           "objective": functionalReport.objective,
           "name": functionalReport.name,
           "input": functionalReport.input,
           "output": functionalReport.output}

In [22]:
write_code_prompt = """You are an expert Python developer tasked with creating Python functions (tools) based on user requests.

            Your process is as follows:
            1. Understand the user's request for a tool (e.g., "tool to send a discord message").
            2. Find relevant Python SDKs for the core task.
            3. See if Composio offers an integration for the relevant service (e.g., 'discord').
            4. Analyze the results:
                - If Composio has an integration, prioritize generating code that utilizes Composio (assume this involves calling a hypothetical 'composio.run_action()' function). Include a comment explaining this choice.
                - If Composio does not have a clear integration, choose the most promising Python SDK found
                - If no suitable SDK is found, state that you cannot create the function.
            5. Generate *only* the complete, runnable Python function code based on your decision.
                - The function should have clear arguments based on the user's likely intent (e.g., for discord, `channel_id` and `message_text`).
                - Include a comprehensive docstring explaining the function, its arguments, and what it returns.
                - Use type hints for all arguments and the return type.
                - If using a standard SDK, add a comment indicating which SDK is intended (e.g., `# Uses discord.py`).
                - If using Composio, structure the function to call `composio.run_action('service_name', 'action_name', params={{...}})` (you'll need to infer 'service_name' and 'action_name' and necessary params). Add comments explaining this structure.
            6. Do not include any explanatory text before or after the code block. Output only the Python code for the function.

    Here are some details about the python function you will be creating:
    <objective>
    {objective}
    </objective>

    <input schema>
    {inputs}
    </input schema>

    <output schema>
    {output}
    </output schema>

    <name of function>
    {name}
    </name of function>

"""

In [26]:

code_snips = ""

def code_production_node(state: FunctionInstructions):
    objective_agent: str = state.objective
    name: str = state.name
    input_args : List[str] = state.input
    output_args: List[str] = state.output
    response = llm.invoke([SystemMessage(content=write_code_prompt.format(
          objective=objective_agent,
          inputs=input_args,
          output=output_args,
          name=name,
    ))])
    code_snips = response
    return {
            "code": response}

In [29]:
workflow = StateGraph(functional_analysis_node)
workflow.add_node("func_analysis", functional_analysis_node)
workflow.add_node("code_write", code_production_node)

workflow.add_edge("code_write", END)
workflow.add_edge("func_analysis","code_write")
workflow.add_edge(START, "func_analysis")
infograph = workflow.compile()


In [28]:
workflow = StateGraph(functional_analysis_node)
workflow.add_node("func_analysis", functional_analysis_node)
# workflow.add_node("code_write", code_production_node)

workflow.add_edge("func_analysis", END)
# workflow.add_edge("func_analysis","code_write")
workflow.add_edge(START, "func_analysis")
infograph = workflow.compile()

In [32]:
import pprint
from langgraph.graph import END, StateGraph, START, MessagesState
from langgraph.prebuilt import ToolNode

from pydantic import BaseModel

class Maintain(BaseModel):
    class Config:
            arbitrary_types_allowed = True
inputs = {
    "messages": [
        ("user", "Create a python function that gets all my emails from gmail and filter them based on senders"),
    ]
}

code_snip = dict()
for output in infograph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
        code_snip = value
    pprint.pprint("\n---\n")

"Output from node 'func_analysis':"
'---'
{ 'input': [ 'gmail_service: object (Gmail API service instance)',
             'senders: list of strings (list of sender email addresses to '
             'filter by)'],
  'name': 'filter_emails_by_sender',
  'objective': 'This Python function connects to a Gmail account using the '
               'Gmail API, retrieves all emails, and filters them based on '
               'specified sender email addresses. It allows users to manage '
               'their inbox by focusing on emails from particular senders.',
  'output': [ 'filtered_emails: list of dictionaries (list of email data '
              'filtered by specified senders)']}
'\n---\n'
"Output from node 'code_write':"
'---'
{ 'code': AIMessage(content='```python\ndef filter_emails_by_sender(gmail_service: object, senders: list[str]) -> list[dict]:\n    """\n    Connects to a Gmail account using the Gmail API, retrieves all emails, \n    and filters them based on specified sender email ad

In [37]:
print(code_snip['code'].content)

```python
def filter_emails_by_sender(gmail_service: object, senders: list[str]) -> list[dict]:
    """
    Connects to a Gmail account using the Gmail API, retrieves all emails, 
    and filters them based on specified sender email addresses.

    Args:
        gmail_service (object): The Gmail API service instance used to interact with the Gmail API.
        senders (list[str]): A list of sender email addresses to filter the emails by.

    Returns:
        list[dict]: A list of dictionaries containing email data filtered by the specified senders.
    """
    filtered_emails = []
    results = gmail_service.users().messages().list(userId='me').execute()
    messages = results.get('messages', [])

    for message in messages:
        msg = gmail_service.users().messages().get(userId='me', id=message['id']).execute()
        headers = msg['payload']['headers']
        for header in headers:
            if header['name'] == 'From':
                sender = header['value']
              

from composio_langchain import ComposioToolSet, App

composio_toolset = ComposioToolSet(api_key='zg6uxxn5rlc3aployj5ddo')

In [26]:
from composio_openai import ComposioToolSet, Action
# Initialize ToolSet (assuming API key is in env)
toolset = ComposioToolSet()
github_tools = toolset.get_tools(apps=[App.GITHUB])
print(f"Fetched {len(github_tools)} tools for GitHub.")

C:\Users\anupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\composio\tools\toolset.py:2042: UserWarning: Running without a Composio API key
  self.workspace.check_for_missing_dependencies(
C:\Users\anupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error

ApiKeyNotProvidedError: API Key not provided, either provide API key or export it as `COMPOSIO_API_KEY` or run `composio login`

In [4]:
from langchain.chains import LLMChain, SimpleSequentialChain, TransformChain
from langchain_community.tools.zapier.tool import ZapierNLARunAction
from langchain_community.utilities.zapier import ZapierNLAWrapper
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [15]:
from bs4 import BeautifulSoup
import html2text
import httpx

def fetch_documents(url: str) -> str:
    """Fetch a document from a URL and return the markdownified text.

    Args:
        url (str): The URL of the document to fetch.

    Returns:
        str: The markdownified text of the document.
    """
    httpx_client = httpx.Client(follow_redirects=True, timeout=10)

    try:
        response = httpx_client.get(url, timeout=10)
        response.raise_for_status()
        html_content = response
        soup = BeautifulSoup(html_content, 'html.parser')
    
        img_tags = soup.find_all('img')
        for img_tag in img_tags:
            img_tag.decompose()

        target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain
        
        if not target_div:
            target_div = soup.find('article') #langraph

        if not target_div:
            target_div = soup.find('html') #langraph

        if not target_div:
            return html2text.html2text(str(soup))
        
        return html2text.html2text(str(target_div))
    except (httpx.HTTPStatusError, httpx.RequestError) as e:
        return f"Encountered an HTTP error: {str(e)}"
    
def fetch_documents_with_links_html(url: str) -> tuple[str, list[tuple[str, str]]]:
    """Fetch a document from a URL, return the markdownified text with links as markdown, and extract links with their titles.

    Args:
        url (str): The URL of the document to fetch.

    Returns:
        tuple[str, list[tuple[str, str]]]: A tuple containing the markdownified text of the document with links, and a list of (link, title) tuples.
    """
    httpx_client = httpx.Client(follow_redirects=True, timeout=10)
    links = []

    try:
        response = httpx_client.get(url, timeout=10)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain

        if not target_div:
            target_div = soup.find('article') #langraph

        if not target_div:
            return "", links # Return empty text but still the links

        # Extract links *before* converting to markdown
        a_tags = target_div.find_all('a')
        for a_tag in a_tags:
            link = a_tag.get('href')
            title = a_tag.get_text(strip=True)
            if link:
                links.append((link, title))

        markdown_converter = html2text.HTML2Text()
        markdown_converter.body_width = 0  # Disable line wrapping for links to stay on one line
        markdown_text = markdown_converter.handle(str(target_div))

        return markdown_text, links
    except (httpx.HTTPStatusError, httpx.RequestError) as e:
        return f"Encountered an HTTP error: {str(e)}", [] # Return error message and empty links
    

In [18]:
doc = fetch_documents("https://python.langchain.com/docs/integrations/tools/arxiv/")
print(doc)

# ArXiv

This notebook goes over how to use the `arxiv` tool with an agent.

First, you need to install the `arxiv` python package.

    
    
    %pip install --upgrade --quiet  langchain-community arxiv  
    
    
    
    from langchain import hub  
    from langchain.agents import AgentExecutor, create_react_agent, load_tools  
    from langchain_openai import ChatOpenAI  
      
    llm = ChatOpenAI(temperature=0.0)  
    tools = load_tools(  
        ["arxiv"],  
    )  
    prompt = hub.pull("hwchase17/react")  
      
    agent = create_react_agent(llm, tools, prompt)  
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  
    

**API Reference:**[hub](https://python.langchain.com/api_reference/langchain/hub/langchain.hub.hub.html) | [AgentExecutor](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent.AgentExecutor.html) | [create_react_agent](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.react

In [19]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)
tools = load_tools(
    ["arxiv"],
)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
def fetch_documents(url: str) -> str:
    """Fetch a document from a URL and return the markdownified text.

    Args:
        url (str): The URL of the document to fetch.

    Returns:
        str: The markdownified text of the document.
    """
    httpx_client = httpx.Client(follow_redirects=True, timeout=10)

    try:
        response = httpx_client.get(url, timeout=10)
        response.raise_for_status()
        html_content = response
        soup = BeautifulSoup(html_content, 'html.parser')
    
        img_tags = soup.find_all('img')
        for img_tag in img_tags:
            img_tag.decompose()

        target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain
        
        if not target_div:
            target_div = soup.find('article') #langraph

        if not target_div:
            target_div = soup.find('html') #langraph

        if not target_div:
            return html2text.html2text(str(soup))
        
        return html2text.html2text(str(target_div))
    except (httpx.HTTPStatusError, httpx.RequestError) as e:
        return f"Encountered an HTTP error: {str(e)}"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate
import json

# 1. Define Tool Specification (within a Python string, for simplicity)
tool_spec = """
{
  "name": "calculate_area",
  "description": "Calculates the area of a rectangle.",
  "input_parameters": [
    {
      "name": "length",
      "type": "number",
      "description": "The length of the rectangle.",
      "required": true
    },
    {
      "name": "width",
      "type": "number",
      "description": "The width of the rectangle.",
      "required": true
    }
  ],
  "output_specification": {
    "type": "number",
    "description": "The area of the rectangle."
  },
  "functionality": {
    "type": "python_function"
  }
}
"""

# 2. Prepare the LLM Agent
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", streaming=True)  # Use a powerful LLM
# (In a full application, you might have other tools)
tools = []


# 3. Create Code Generation Prompt
prompt_template = PromptTemplate.from_template(
    """
You are a Python programmer. You need to write a Python function
that implements the following tool:

Tool Specification:
{tool_specification}

-   Write a complete, runnable Python function.
-   Include a docstring that explains what the function does and how to use it.
-   Use type hints.
-   Do not include any code outside of the function definition.
"""
)
prompt = prompt_template.format(tool_specification=tool_spec)

# 4. Agent Generates Code
s = llm.invoke([SystemMessage(content=prompt)])
print("Generated Code:")
print(s.content)

Generated Code:
```python
def calculate_area(length: float, width: float) -> float:
    """
    Calculates the area of a rectangle.

    Parameters:
    length (float): The length of the rectangle.
    width (float): The width of the rectangle.

    Returns:
    float: The area of the rectangle, calculated as length * width.

    Example:
    >>> calculate_area(5, 3)
    15.0
    """
    return length * width
```


In [ ]:
"""You are an expert Python developer tasked with creating Python functions (tools) based on user requests.

            Your process is as follows:
            1. Understand the user's request for a tool (e.g., "tool to send a discord message").
            2. Find relevant Python SDKs for the core task.
            3. See if Composio offers an integration for the relevant service (e.g., 'discord').
            4. Analyze the results:
                - If Composio has an integration, prioritize generating code that utilizes Composio (assume this involves calling a hypothetical 'composio.run_action()' function). Include a comment explaining this choice.
                - If Composio does not have a clear integration, choose the most promising Python SDK found
                - If no suitable SDK is found, state that you cannot create the function.
            5. Generate *only* the complete, runnable Python function code based on your decision.
                - The function should have clear arguments based on the user's likely intent (e.g., for discord, `channel_id` and `message_text`).
                - Include a comprehensive docstring explaining the function, its arguments, and what it returns.
                - Use type hints for all arguments and the return type.
                - If using a standard SDK, add a comment indicating which SDK is intended (e.g., `# Uses discord.py`).
                - If using Composio, structure the function to call `composio.run_action('service_name', 'action_name', params={{...}})` (you'll need to infer 'service_name' and 'action_name' and necessary params). Add comments explaining this structure.
            6. Do not include any explanatory text before or after the code block. Output only the Python code for the function.
"""

'You are an expert Python developer tasked with creating Python functions (tools) based on user requests.\n\n            Your process is as follows:\n            1. Understand the user\'s request for a tool (e.g., "tool to send a discord message").\n            2. Use the \'sdk_search_tool\' to find relevant Python SDKs for the core task.\n            3. Use the \'composio_check_tool\' to see if Composio offers an integration for the relevant service (e.g., \'discord\').\n            4. Analyze the results:\n                - If Composio has an integration, prioritize generating code that utilizes Composio (assume this involves calling a hypothetical \'composio.run_action()\' function). Include a comment explaining this choice.\n                - If Composio does not have a clear integration, choose the most promising Python SDK found in the search results.\n                - If no suitable SDK is found, state that you cannot create the function.\n            5. Generate *only* the com

In [ ]:
desc->check if any specific tool is good-> fetch doc link-> execute

https://langchain-ai.github.io/langgraph/how-tos/update-state-from-tools/ 
tool command handling 
